<a href="https://colab.research.google.com/github/lizaoh/smp_program_data/blob/main/smp2018_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of Script

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pymupdf
!pip install pymupdf-layout
!pip install pymupdf4llm
!pip install wordfreq
# !pip install rapidfuzz
import glob
import os
import pathlib
import pymupdf
import pymupdf.layout
import pymupdf4llm
import re
import pandas as pd
import unicodedata
import wordfreq
# from rapidfuzz import process, fuzz

In [4]:
pdfs_path = '/content/drive/MyDrive/math_psych_work/Conference Programs/'

# Functions
Created with help from GPT 5.2, but some are my own code just turned into a function.

In [5]:
def clean_text(text, words_to_hyphenate=None):
    if not text:
        return text

    text = re.sub(r'-\s+(?!\b(?:and|or)\b)', '-', text)  # Get rid of - and space after
                                                        # unless word after is
                                                        # "and" or "or"

    text = text.strip()
    text = fix_ligatures(text)

    return text

In [6]:
LIGATURE_MAP = {
    "ﬁ": "fi", "ﬂ": "fl", "ﬃ": "ffi", "ﬄ": "ffl", "ﬀ": "ff", "ﬅ": "ft", "ﬆ": "st",
    "Æ": "ffi", "¨u": "ü", "¨a": "ä", "´e": "é", "`e": "è", "`a": "à", "¨o": "ö",
    "˚a": "å", "c¸": "ç", '“': '"', '”': '"', "’": "'", '˜n': 'ñ', 'ˇs': 'š',
    "âĂŸ": "'", "``": '"', "↵": "ff", "✏": "ffl"
}

def fix_ligatures(text):
    # Replace known ligatures
    for bad, good in LIGATURE_MAP.items():
        text = text.replace(bad, good)

    # Replace any private-use ligature (common in PDFs)
    cleaned_chars = []
    for ch in text:
        name = unicodedata.name(ch, "")
        if "LIGATURE" in name.upper():
            # Try to break it apart: remove spaces and lowercase
            base = name.split("LIGATURE")[-1]
            base = base.replace(" ", "").lower()
            cleaned_chars.append(base)
        else:
            cleaned_chars.append(ch)

    return "".join(cleaned_chars)

In [28]:
def normalize_title_prefix(title, n_words=5):
    words = title.lower().split()
    return ' '.join(words[:n_words])

# Program

149 entries total (4 plenary talks and 145 other talks)

 No abstracts here, just schedule



## Grab text from the pdf

In [9]:
year = '2018'
program = pymupdf.open(pdfs_path + f'smp{year}_schedule.pdf')

In [10]:
program_text = []

for page in program[1:]:  # first page is just workshops
  program_text.append(page.get_text('text'))

In [43]:
program_text[0]

'Date: Sunday, 22/Jul/2018\n8:00am\n-\n9:40am\nNSF Decision 1:\nProbabilistic Specification\nand Quantitative Testing of\nDecision Theories\nLocation: Great Hall\n8:00am - 8:20am\nCognitive Aging and\nTests of Rationality\nClintin Phillip Davis-Stober,\nSanghyuk Park, Michel\nRegenwetter\n8:20am - 8:40am\nMathematical modeling\nof sexual decision\nmaking strategies\nLaura Elizabeth Hatz,\nKayleigh Noelle McCarty,\nSanghyuk Park, Denis\nMichael McCarthy, Clintin P\nDavis-Stober\n8:40am - 9:00am\nRational mate choice in\nfruit flies? A reanalysis\nBryanna Rose Fields,\nMichel Regenwetter, Bart\nSmeulders, Clintin Davis-\nStober, Cihang Wang\n9:00am - 9:20am\nTesting axioms of\nstochastic discrete\nchoice using population\nchoice probabilities\nWilliam James\nMcCausland, Clintin Davis-\nStober, Anthony Marley\n9:20am - 9:40am\nInferring learning and\nadaptivity in multi-\nattribute decisions\nusing novel probabilistic\nspecifications of\nheuristics, and well-\ndefined error models\nPercy 

## Split text into presentation entries

In [44]:
initial_entries = []
for page_text in program_text:
  initial_split = re.split(r'\n\d{1,2}:\d{2}(?:a|p)m\s-\s\d{1,2}:\d{2}(?:a|p)m\n', page_text)
  initial_entries.extend(initial_split)

In [45]:
not_talk_words = [
    'Coffee', 'Lunch', 'Dinner', 'Meeting'
]
filtered_entries = [entry for entry in initial_entries if not any(word in entry for word in not_talk_words)]

In [48]:
filtered_entries[2:7]

['Cognitive Aging and\nTests of Rationality\nClintin Phillip Davis-Stober,\nSanghyuk Park, Michel\nRegenwetter',
 'Mathematical modeling\nof sexual decision\nmaking strategies\nLaura Elizabeth Hatz,\nKayleigh Noelle McCarty,\nSanghyuk Park, Denis\nMichael McCarthy, Clintin P\nDavis-Stober',
 'Rational mate choice in\nfruit flies? A reanalysis\nBryanna Rose Fields,\nMichel Regenwetter, Bart\nSmeulders, Clintin Davis-\nStober, Cihang Wang',
 'Testing axioms of\nstochastic discrete\nchoice using population\nchoice probabilities\nWilliam James\nMcCausland, Clintin Davis-\nStober, Anthony Marley',
 'Inferring learning and\nadaptivity in multi-\nattribute decisions\nusing novel probabilistic\nspecifications of\nheuristics, and well-\ndefined error models\nPercy K Mistry, Jennifer S.\nTrueblood\nICCM-1: ICCM I\nLocation: State Room']

## Sort authors, affiliations, title, and abstract

In [15]:
parsed_entries = []

for entry in filtered_entries:
  split_entry = entry.split('\n')
  idx = next((i for i, line in enumerate(split_entry) if ',' in line), None)
  if idx:
    title = ' '.join(split_entry[:idx])
    authors = ' '.join(split_entry[idx:])
  else:
    title = ' '.join(split_entry)
    authors = ''

  cleaned_title = clean_text(title)
  cleaned_authors = clean_text(authors)

  parsed_entries.append({
      'year': year,
      'author(s)': cleaned_authors,
      'affiliation(s)': '',
      'title': cleaned_title,
      'type': '',
      'abstract': ''
  })

In [49]:
parsed_entries[2:7]

[{'year': '2018',
  'author(s)': 'Clintin Phillip Davis-Stober, Sanghyuk Park, Michel Regenwetter',
  'affiliation(s)': '',
  'title': 'Cognitive Aging and Tests of Rationality',
  'type': '',
  'abstract': ''},
 {'year': '2018',
  'author(s)': 'Laura Elizabeth Hatz, Kayleigh Noelle McCarty, Sanghyuk Park, Denis Michael McCarthy, Clintin P Davis-Stober',
  'affiliation(s)': '',
  'title': 'Mathematical modeling of sexual decision making strategies',
  'type': '',
  'abstract': ''},
 {'year': '2018',
  'author(s)': 'Bryanna Rose Fields, Michel Regenwetter, Bart Smeulders, Clintin Davis-Stober, Cihang Wang',
  'affiliation(s)': '',
  'title': 'Rational mate choice in fruit flies? A reanalysis',
  'type': '',
  'abstract': ''},
 {'year': '2018',
  'author(s)': 'McCausland, Clintin Davis-Stober, Anthony Marley',
  'affiliation(s)': '',
  'title': 'Testing axioms of stochastic discrete choice using population choice probabilities William James',
  'type': '',
  'abstract': ''},
 {'year': '2

# ICCM Proceedings

I found this document online with some abstracts and papers from the ICCM parts of the conference, so we can fill in some of the missing abstracts.

Not all papers here are in the program, and not all have abstracts. If they do have an abstract, it's located before the keywords.

There are 45 total papers here.

In [17]:
proceedings = pymupdf.open(pdfs_path + 'iccm2018_proceedings.pdf')

In [18]:
proceedings_text = pymupdf4llm.to_markdown(proceedings) # errors are from figures

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations



In [57]:
proceedings_text[12_800:14_000]

'e text -----**<br>\n\n\nvii \n\nICCM2018 \n\n# **An extensible framework for mechanistic processing models: From representational linguistic theories to quantitative model comparison** \n\n**Adrian Brasoveanu (abrsvn@gmail.com)** Department of Linguistics, 1156 High Street Santa Cruz, CA 95065, USA \n\n**Jakub Dotlaˇcil (j.dotlacil@gmail.com)** ILLC, Science Park 107 1098 XG Amsterdam, The Netherlands \n\n## **Abstract** \n\nWe introduce a Python3 reimplementation of ACTR (Anderson and Lebiere 1998, Anderson 2007) in which we build an end-to-end simulation of syntactic parsing in a typical self-paced reading experiment. The model uses an eager left-corner parsing strategy implemented as a skill in procedural memory (following Lewis and Vasishth 2005), makes use of independently motivated components of the ACT-R framework (procedural memory, contentaddressable declarative memory – cf. Wagers et al. 2009), and explicitly models the motor and visual processes involved in self-paced readi

## Split text into talk entries

In [20]:
all_papers = proceedings_text.split('\nvii \n\nICCM2018 \n\n# ')[1]

In [21]:
references_split = all_papers.split('**References**')[:-1]

In [23]:
abstract_entries = []
for entry in references_split:
  if '**Abstract**' in entry:
    # Remove rest of paper after abstract
    entry_info = re.split(r'\*{2}Keywords:\*{0,2}', entry)[0].strip()
    if 'ICCM2018' in entry_info:
      # Remove references from previous entry
      entry_info = re.split(r'\n\nICCM2018 \n\n##? (?=\*)', entry_info)[1]
    abstract_entries.append(entry_info)
  else:
    continue

In [59]:
abstract_entries[:2]

['**An extensible framework for mechanistic processing models: From representational linguistic theories to quantitative model comparison** \n\n**Adrian Brasoveanu (abrsvn@gmail.com)** Department of Linguistics, 1156 High Street Santa Cruz, CA 95065, USA \n\n**Jakub Dotlaˇcil (j.dotlacil@gmail.com)** ILLC, Science Park 107 1098 XG Amsterdam, The Netherlands \n\n## **Abstract** \n\nWe introduce a Python3 reimplementation of ACTR (Anderson and Lebiere 1998, Anderson 2007) in which we build an end-to-end simulation of syntactic parsing in a typical self-paced reading experiment. The model uses an eager left-corner parsing strategy implemented as a skill in procedural memory (following Lewis and Vasishth 2005), makes use of independently motivated components of the ACT-R framework (procedural memory, contentaddressable declarative memory – cf. Wagers et al. 2009), and explicitly models the motor and visual processes involved in self-paced reading. The ACTR model can be embedded in a Bayesi

## Split entries into title, author / affiliation, abstract

In [25]:
abstracts_to_match = []

for entry in abstract_entries:
  cleaned_entry = clean_text(entry)

  # Splits title, authors, and affiliations from abstract
  info_text, abstract = cleaned_entry.split('**Abstract**')
  abstract = abstract.strip()

  # Gets title (authors are bolded but always have email in parentheses)
  bolded_text = re.findall(r'\*\*(.*?)\*\*', info_text, flags=re.DOTALL)

  title_parts = []
  for b in bolded_text:
      if '@' in b:
          break
      title_parts.append(b)

  title = ' '.join(title_parts)

  # Removes title to get authors and affiliations
  auth_and_aff = info_text

  for part in title_parts:
      auth_and_aff = auth_and_aff.replace(f'**{part}**', '', 1)

  auth_and_aff = auth_and_aff.strip()

  entry_parts = {
      'title': title,
      'auth_and_aff': auth_and_aff,
      'abstract': abstract
  }

  abstracts_to_match.append(entry_parts)

In [60]:
abstracts_to_match[:2]

[{'title': 'An extensible framework for mechanistic processing models: From representational linguistic theories to quantitative model comparison',
  'auth_and_aff': '**Adrian Brasoveanu (abrsvn@gmail.com)** Department of Linguistics, 1156 High Street Santa Cruz, CA 95065, USA \n\n**Jakub Dotlaˇcil (j.dotlacil@gmail.com)** ILLC, Science Park 107 1098 XG Amsterdam, The Netherlands \n\n##',
  'abstract': 'We introduce a Python3 reimplementation of ACTR (Anderson and Lebiere 1998, Anderson 2007) in which we build an end-to-end simulation of syntactic parsing in a typical self-paced reading experiment. The model uses an eager left-corner parsing strategy implemented as a skill in procedural memory (following Lewis and Vasishth 2005), makes use of independently motivated components of the ACT-R framework (procedural memory, contentaddressable declarative memory – cf. Wagers et al. 2009), and explicitly models the motor and visual processes involved in self-paced reading. The ACTR model can 

# Match ICCM proceedings to program talks and add abstracts

Matching based on first 5 words because some titles from the program have an author at the end of it.

In [27]:
program_titles = [entry['title'] for entry in parsed_entries]
iccm_titles = [entry['title'] for entry in abstracts_to_match]

In [38]:
# Gets first 5 words from the program titles
program_prefixes = [
    normalize_title_prefix(t)
    for t in program_titles
]

for t in iccm_titles:
    prefix = normalize_title_prefix(t)
    if prefix in program_prefixes:
        title_index = program_prefixes.index(prefix)
        parsed_entries[title_index]['abstract'] = abstracts_to_match[iccm_titles.index(t)]['abstract']

In [62]:
parsed_entries[7:12]

[{'year': '2018',
  'author(s)': 'Can Serif Mekik, Ron Sun, David Yun Dai',
  'affiliation(s)': '',
  'title': "Similarity-based and Rule-based Reasoning in Raven's Matrices",
  'type': '',
  'abstract': "Raven's Matrices are a well-known family of intelligence tests. Based on an analysis of a recently proposed similarity-based approach, we present a new analysis of performance in the task. This analysis represents an alternative to the dominant rule induction approach. In particular, it suggests that similaritybased and rule-based reasoning may synergistically enable performance. We discuss possible complementary roles of the two types of reasoning, suggest possible explanations of observed empirical phenomena, and compare and contrast our model with other prominent models of the task."},
 {'year': '2018',
  'author(s)': 'Nicolas Oliver Riesterer, Marco Ragni',
  'affiliation(s)': '',
  'title': 'The Implications of Guessing Types in Multinomial Processing Tree Models: Conditional Rea

# Create df and convert to csv

In [63]:
df = pd.DataFrame(parsed_entries, columns=["year", "author(s)", "affiliation(s)", "title", "type", "abstract"])

In [66]:
df[7:12]

,year,author(s),affiliation(s),title,type,abstract
7,2018,"Can Serif Mekik, Ron Sun, David Yun Dai",,Similarity-based and Rule-based Reasoning in R...,,Raven's Matrices are a well-known family of in...
8,2018,"Nicolas Oliver Riesterer, Marco Ragni",,The Implications of Guessing Types in Multinom...,,
9,2018,"Kevin Zish, Nathan Aguiar, Malcolm McCurry, Er...",,The time course recovery of confidence judgmen...,,Participants were interrupted during a procedu...
10,2018,"John K. Lindstedt, Michael Byrne",,Simple agglomerative visual grouping for ACT-R,,The human visual system tends to group related...
11,2018,"Joseph Glavan, Joseph Houpt Methodology: Metho...",,An Integrated Working Memory Model For Time-Ba...,,The time-based resource-sharing (TBRS) model e...


In [65]:
df.to_csv(f"/content/drive/MyDrive/math_psych_work/csv/smp{year}_program.csv", index=False)